In [2]:
##ESTABLISH CONNECTION
import wrds
db = wrds.Connection(wrds_username='stevesheridan')
db.create_pgpass_file()

Loading library list...
Done
Enter your WRDS username [stevesheridan]:stevesheridan
Enter your password:········


In [3]:
### READ IN CSV ROW (LOOP)
import csv
import pandas
from datetime import datetime

def priceChangePercent(entryPrice, price_data, endTime, percentChange):
    if endTime.time() == "20:00:00.000000":
        return percentChange
    else:
        endPrice = -1
        i = 0
        times = [None]*100000000
        prices = [None]*100000000
        rows = price_data.shape[0]

        price_data = price_data.reset_index()  # make sure indexes pair with number of rows
        for index, row in price_data.iterrows():
            times[i] = row['time_m']
            prices[i] = row['price']
            #print(f"Times[i]: {times[i]}")
            if i == 0:
                i += 1
                continue
            #print(f"Times[i-1]: {times[i-1]}")
            if endTime.time() <= times[i] and endTime.time() >= times[i-1]:
                endPrice = prices[i-1]
                #print(f"Closest time to {endTime.time()} is {times[i-1]}")
                #print(f"The price at {times[i-1]} was {endPrice}")
                break
            elif endTime.time() >= times[i] and i == (rows-1):
                #print("REACHED END OF DATAFRAME")
                return percentChange
            i += 1
        
        if entryPrice != -1:
            #print("IN ENTRYPRICE IF STATEMENT")
            print(f"entryPrice: {entryPrice}")
            print(f"endPrice: {endPrice}")
            if endPrice == -1:
                totalRetJaunt = -1000
                return totalRetJaunt
            else:
                totalChange = ((endPrice - entryPrice)/entryPrice)*100
                totalChange = round(totalChange, 2)
            return totalChange
        else:
            return percentChange
        

def correctLowTrades(price_data):
    times = [None]*100000000
    i = 0

    price_data = price_data.reset_index()  # make sure indexes pair with number of rows
    for index, row in price_data.iterrows():
        times[i] = row['time_m']
        if i == 0:
            i += 1
            continue
        if times[i] < times[i-1]:
            #print('HERE')
            #print(type(row['time_m']))
            #print(i)
            price_data = price_data[0:i]
            #print(price_data)
            break
        i += 1
    return price_data


def getPriceBeforeNews(price_data, starttime):
    i = 0
    times = [None]*100000000
    prices = [None]*100000000
    rows = price_data.shape[0]
    endPrice = -1

    price_data = price_data.reset_index()  # make sure indexes pair with number of rows
    for index, row in price_data.iterrows():
        times[i] = row['time_m']
        prices[i] = row['price']
        if i == 0:
            i += 1
            continue
        if starttime.time() <= times[i] and starttime.time() >= times[i-1]:
            endPrice = prices[i-1]
            print(f"Closest time to {starttime.time()} is {times[i-1]}")
            print(f"The price at {times[i-1]} was {endPrice}")
            break
        elif starttime.time() >= times[i] and i == (rows-1):
            print("REACHED END OF DATAFRAME")
            return -1
        i += 1
    return endPrice
    

def GetNbboAtNews(price_data, starttime):
    i = 0
    times = [None]*100000000
    prices = [None]*100000000
    bids = [None]*100000000
    asks = [None]*100000000
    rows = price_data.shape[0]

    if rows > 1:
        price_data = price_data.reset_index()  # make sure indexes pair with number of rows
        for index, row in price_data.iterrows():
            times[i] = row['time_m']
            bids[i] = row['bid']
            asks[i] = row['ask']

            if i == 0:
                i += 1
                continue
            if starttime.time() <= times[i] and starttime.time() >= times[i-1]:
                bid = bids[i]
                ask = asks[i]
                print(f"Closest time to {starttime.time()} is {times[i]}")
                print(f"The bid at {times[i]} was {bid} and the ask was {ask}")
                break
            elif starttime.time() >= times[i] and i == (rows-1):
                print("REACHED END OF DATAFRAME")
                return [-1, -1]
            i += 1
    else:
        bid = -1
        ask = -1
    
    nbbos = [bid, ask]
    return nbbos    
    
#######################################################################################
#######################################################################################
#######################################################################################
#######################################################################################
#######################################################################################
#######################################################################################

price_data = pandas.DataFrame()

with open('result.csv', 'w', newline = '', encoding = "utf8") as csvfile3:
    with open('headlines.csv', 'r', newline = '', encoding = "utf8") as csvfile2:
        sourcer = csv.reader(csvfile2, delimiter = ',', quotechar = '|')
        writer = csv.writer(csvfile3, delimiter = ',', quotechar = '|', quoting = csv.QUOTE_MINIMAL)
        top_row = ["Ticker", "Date", "Time", "Headline", "Market Cap", "1min Change", "5min Change", "15min Change", "1hr Change", "2hr Change", "4hr Change", "EOD Change", "Price Before News", "Bid at News", "Ask at News", "Percent Dif Prc B4 vs Bid"]
        writer.writerow(top_row)
        
##GET DATE, TIME, AND TICKER        
        for row in sourcer:
            ticker = row[0]
            datetimeOne = row[1]
            headline = row[2]
            #print(f"Ticker: {ticker}    Datetime: {datetimeOne}    Headline: {headline}")
            
            date = datetimeOne.split("T")[0]
            time = datetimeOne.split("T")[1]
            #time += '.000000'
            hour = time.split(":")[0]
            minute = time.split(":")[1]
            second = time.split(":")[2]
            second = second[:-3]
            #print(f"Date: {date}    Time: {time}")
            
            year = date.split("-")[0]
            month = date.split("-")[1]
            day = date.split("-")[2]
            #print(f"Year: {year}   Month: {month}    Day: {day}")
                             
##GET DATABASE STRING USING DATE FOR TRADES
            library = "taqm_" + year
            table = ".ctm_"+ year + month + day
            databaseName = library + table
            #print(databaseName)

##GET DATABASE STRING USING DATE FOR QUOTES
            nbboLibrary = "taqm_" + year
            nbboTable = ".cqm_"+ year + month + day
            nbboDatabaseName = nbboLibrary + nbboTable
            #print(nbboDatabaseName)
            
##GET START AND END TIMES FOR PRICE DATA
            #startHour = int(hour) - 1
            starttime = f"{hour}:{minute}:{second}.000000"
            #print(starttime)

##MAKE SQL DATA CALL FOR CRSP DATA
            names_data = db.raw_sql("select permno, ticker, comnam, date, nameendt from crsp.mseall")
            #display(names_data)
    
            curDate = f"{year}{month}{day}"
            curDate = int(curDate)
            #print(curDate)

            call_date = f"{month}/{day}/{year}"
            
            ticker = str(ticker)
            ticker = f'{ticker}'
            ticker = str(ticker)
            ticker = f'{ticker}'
            ticker = str(ticker)
            ticker = ticker.replace("\ufeff", "")
            #ticker = 'AAPL'
            #print(ticker)
            
            permno = 0
            
            names_data = names_data.reset_index()  # make sure indexes pair with number of rows
            for index, row in names_data.iterrows():
                if ticker == row['ticker']:
                    #print('YES')
                    date2 = row['nameendt']
                    date2 = str(date2)
                    date2 = date2.replace("-", "")
                    if date2 == "None":
                        continue
                    dateNumber = int(date2)
                    #print(f"dateNumber: {dateNumber} -- curDate: {curDate}")
                    if dateNumber >= curDate:
                        permno = row['permno']
                        #print(permno)
                        break

            #print(f"The PERMNO of {ticker} is {permno}")
            
            if(permno != 0):
                query_market_cap = f"select permno, date, prc, shrout from crsp.dsf where permno = {permno} and date = '{call_date}'"
                market_cap_data = db.raw_sql(query_market_cap)
                #display(market_cap_data)
                
                rows_cap = market_cap_data.shape[0]
                
                if rows_cap > 0:
                    price = market_cap_data.loc[0].at["prc"]
                    shares_out = market_cap_data.loc[0].at["shrout"]

                    market_cap = abs(price) * shares_out
                    market_cap = round(market_cap, 0)
                    #print(f"The market cap of {ticker} on {call_date} was {market_cap}")
                else:
                    market_cap = -1
                    #print(f"No market cap data for {ticker}") 
            else:
                market_cap = -1
                #print(f"No market cap data for {ticker}")

##MAKE SQL DATA CALL FOR TAQ DATA
            query = f"select date, time_m, sym_root, size, price, ex, tr_source, tr_rf from {databaseName} where sym_root = '{ticker}' and time_m >= '{starttime}'"
            total_query = f"select date, time_m, sym_root, size, price, ex, tr_source, tr_rf from {databaseName} where sym_root = '{ticker}' "
            nbbo_query = f"select date, time_m, sym_root, bid, bidsiz, ask, asksiz from {nbboDatabaseName} where sym_root = '{ticker}'"
            
            total_price_data = db.raw_sql(total_query)
            price_data = db.raw_sql(query)
            nbbo_data = db.raw_sql(nbbo_query)
            display(price_data)
            display(total_price_data)
            display(nbbo_data)

##CORRECT FOR RANDOM SUPER LOW TRADES   
            total_price_data = correctLowTrades(total_price_data)
            nbbo_data = correctLowTrades(nbbo_data)
            price_data = correctLowTrades(price_data)
    
##GET PRICE RIGHT AT TIME OF NEWS
            rows = price_data.shape[0]
            
            if rows > 0:
                startPrice = abs(price_data.loc[0].at["price"])
                endPrice = abs(price_data.loc[rows-1].at["price"])
                percentChange = ((endPrice - startPrice)/startPrice)*100
                percentChange = round(percentChange, 2)
                #print(f"Price change for {ticker} on {date} was {percentChange}%")
            else:
                percentChange = -1000.0
                #print(f"No price data for {ticker} on {date}")
                
##GET PRICE BEFORE NEWS
            starttimeOb = datetime.strptime(starttime, '%H:%M:%S.%f')
            priceBeforeNews = getPriceBeforeNews(total_price_data, starttimeOb)
            print(f"Price before news was {priceBeforeNews}")
            
##GET NBBO AT TIME OF NEWS
            nbboAtNews = GetNbboAtNews(nbbo_data, starttimeOb)
            bidAtNews = nbboAtNews[0]
            askAtNews = nbboAtNews[1]
            percentDif = (abs(priceBeforeNews - bidAtNews)/((priceBeforeNews + bidAtNews) / 2))*100
            print(f"Bid: {bidAtNews}, Percent Dif: {percentDif}")

##PROCESS PRICE DATA
            trades = [None] * 100000000
            u = 0
            hourInt = int(hour)
            minInt = int(minute)
            secInt = int(second)
            
##1min ahead
            if minInt < 59:
                newMinInt = minInt + 1
                if newMinInt <=9:
                    oneMinAhead = f"{hour}:0{newMinInt}:{second}.000000"
                else:
                    oneMinAhead = f"{hour}:{newMinInt}:{second}.000000"
            else:
                newMinInt = 0
                newHourInt = hourInt + 1
                if newHourInt >= 20:
                    oneMinAhead = "20:00:00.000000"
                oneMinAhead = f"{newHourInt}:0{newMinInt}:{second}.000000"

##5min ahead
            if minInt < 55:
                newMinInt = minInt + 5
                if newMinInt <=9:
                    fiveMinAhead = f"{hour}:0{newMinInt}:{second}.000000"
                else:
                    fiveMinAhead = f"{hour}:{newMinInt}:{second}.000000"
            else:
                newMinInt = (minInt + 5) - 60
                newHourInt = hourInt + 1
                if newHourInt >= 20:
                    fiveMinAhead = "20:00:00.000000"
                if newMinInt <= 9:
                    fiveMinAhead = f"{newHourInt}:0{newMinInt}:{second}.000000"
                else:
                    fiveMinAhead = f"{newHourInt}:{newMinInt}:{second}.000000"            

##15min ahead
            if minInt < 45:
                newMinInt = minInt + 15
                if newMinInt <=9:
                    fifteenMinAhead = f"{hour}:0{newMinInt}:{second}.000000"
                else:
                    fifteenMinAhead = f"{hour}:{newMinInt}:{second}.000000"
            else:
                newMinInt = (minInt + 15) - 60
                newHourInt = hourInt + 1
                if newHourInt >= 20:
                    fifteenMinAhead = "20:00:00.000000"
                if newMinInt <=9:
                    fifteenMinAhead = f"{newHourInt}:0{newMinInt}:{second}.000000"
                else:
                    fifteenMinAhead = f"{newHourInt}:{newMinInt}:{second}.000000"
                        
##1hr ahead
            newHourInt = hourInt + 1
            if newHourInt >= 20:
                hourAhead = "20:00:00.000000"
            else:
                hourAhead = f"{newHourInt}:{minute}:{second}.000000"

##2hr ahead
            newHourInt = hourInt + 2
            if newHourInt >= 20:
                twoHourAhead = "20:00:00.000000"
            else:
                twoHourAhead = f"{newHourInt}:{minute}:{second}.000000"

##4hr ahead
            newHourInt = hourInt + 4
            if newHourInt >= 20:
                fourHourAhead = "20:00:00.000000"
            else:
                fourHourAhead = f"{newHourInt}:{minute}:{second}.000000"
                        
            #print(f"Starttime: {starttime}")
            #print(f"1min: {oneMinAhead}")
            #print(f"5min: {fiveMinAhead}")
            #print(f"15min: {fifteenMinAhead}")
            #print(f"1hr: {hourAhead}")
            #print(f"2hr: {twoHourAhead}")
            #print(f"4hr: {fourHourAhead}")
            
##DATETIME OBJECTS
            oneMinAheadOb = datetime.strptime(oneMinAhead, '%H:%M:%S.%f')
            fiveMinAheadOb = datetime.strptime(fiveMinAhead, '%H:%M:%S.%f')
            fifteenMinAheadOb = datetime.strptime(fifteenMinAhead, '%H:%M:%S.%f')
            hourAheadOb = datetime.strptime(hourAhead, '%H:%M:%S.%f')
            twoHourAheadOb = datetime.strptime(twoHourAhead, '%H:%M:%S.%f')
            fourHourAheadOb = datetime.strptime(fourHourAhead, '%H:%M:%S.%f')
            
            #print(f"{oneMinAheadOb}\n{fiveMinAheadOb}\n{fifteenMinAheadOb}\n{hourAheadOb}\n{twoHourAheadOb}\n{fourHourAheadOb}")

##GET PRICE CHANGE DATA FROM DATAFRAME
            if rows > 0:
                entryPrice = abs(price_data.loc[0].at["price"])          
                oneMinChange = priceChangePercent(entryPrice, price_data, oneMinAheadOb, percentChange)
                fiveMinChange = priceChangePercent(entryPrice, price_data, fiveMinAheadOb, percentChange)
                fifteenMinChange = priceChangePercent(entryPrice, price_data, fifteenMinAheadOb, percentChange)
                hourChange = priceChangePercent(entryPrice, price_data, hourAheadOb, percentChange)
                twoHourChange = priceChangePercent(entryPrice, price_data, twoHourAheadOb, percentChange)
                fourHourChange = priceChangePercent(entryPrice, price_data, fourHourAheadOb, percentChange)
            else:
                oneMinChange = -1000
                fiveMinChange = -1000
                fifteenMinChange = -1000
                hourChange = -1000
                twoHourChange = -1000
                fourHourChange = -1000

##WRITE PRICE DATA TO CSV COLUMNS
            backtest_results = [ticker, date, starttime, headline, market_cap, oneMinChange, fiveMinChange, fifteenMinChange, hourChange, twoHourChange, fourHourChange, percentChange, priceBeforeNews, bidAtNews, askAtNews, percentDif]
            print(backtest_results)
            writer.writerow(backtest_results)

KeyboardInterrupt: 

In [ ]:
db.close()